In [1]:
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np

# Helper libraries
from w266_common import utils, vocabulary

# Prelude: working with raw text data

You're probably accustomed to a typical machine learning set-up, where each example consists of a (fixed) set of features $x^{(i)}$ and an associated target $y^{(i)}$. _(Note: when there's any ambiguity, we'll use superscripts like $x^{(i)}$ to refer to the $i^{th}$ example in the dataset, and subscripts like $x_j$ to refer to the $j^{th}$ feature or vector element.)_

If we have $d$ features, we might have $x \in \mathbb{R}^{d}$, and in a $k$-way classification problem, we have $y \in \{0,1,\ldots,k - 1\}$. Some of our features might be indicators, $x_j \in \{0,1\}$, and some subset of our features might be sparse - in that most of them are zero, most of the time.

Of course, text data doesn't start in this form. Instead, what we get is a sentence, and we'll need to apply a few steps to convert it into a representation suitable for machine learning. As an example, let's consider sentiment analysis, where our labels $y \in \{0,1\}$ represent negative or positive sentiment. A training example might be:

In [2]:
x_text, y = ("I love W266!", 1)

We don't need to do much to $y$, but to get $x$ ready for a classifier we need to:
1. Tokenize the text into individual words (tokens)
2. Canonicalize the tokens
3. Convert the tokens to a sequence of integer IDs
4. (optional) Convert the IDs to a feature vector

The integer IDs are often referred to as _word ids_ or _token ids_. These might be familiar from live session, when we converted the word "the" into a column index in an embedding table. (In another sense, indices are just a representation of a one-hot vector.)

### Tokenization

This can be as simple as calling `string.split()` - good enough for English and many European languages - but we could also do something more sophisticated here. For now, let's just use the Penn Treebank tokenizer, which uses a set of regular expressions to do things like separate punctuation:

In [3]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
x_tokens = tokenizer.tokenize(x_text)
x_tokens

### Canonicalization

Depending on the application, we might want to do some pre-processing to remove spurious variation in the text. For example, we might want to lowercase words to avoid storing separate features for "I" and "i", and we might want to replace numbers with a special token rather than keep track of every possible value.

We've implemented the above transformation in `utils.canonicalize_word`:

In [4]:
print("Example: '$1.50' -> '{:s}'".format(utils.canonicalize_word("$1.50")))
print("Example: 'FooBar' -> '{:s}'".format(utils.canonicalize_word("FooBar")))
x_tokens_canonical = utils.canonicalize_words(x_tokens)
x_tokens_canonical

### Conversion to IDs

While there are a few ML models that operate directly on strings - you'll implement two of them in Assignment 5 - in most cases (and *always* for neural networks) you'll need to convert the tokens to integer IDs that can index into a feature vector. To do this, we'll need to keep track of a **vocabulary**, which in its simplest form is just a dictionary:

In [5]:
vocab_dict = {"!": 0, "i": 1, "love": 2, "w266": 3}
x_ids = [vocab_dict[token] for token in x_tokens_canonical]
x_ids

Of course, in most cases the corresponding feature vector has a fixed size, and occasionally you'll encounter words that the model has never seen before. Typically, we'll replace such words with a special token, `<unk>`.

In the example below, note that the third token gets ID 0, corresponding to `<unk>`:

In [6]:
# Suppose we don't know about the word 'w266'
vocab_dict = {w:i for i, w in enumerate(["<unk>", "!", "i", "love"])}
print(vocab_dict)
x_ids = [vocab_dict.get(token, vocab_dict['<unk>']) for token in x_tokens_canonical]
x_ids

For most of this class, we'll use a `Vocabulary` object to manage this conversion. We've implemented one in the `w266_common.vocabulary` module, and most NLP packages will have something that works similarly.

Here's how to use it:

In [7]:
vocab = vocabulary.Vocabulary(x_tokens_canonical, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)
x_ids = vocab.words_to_ids(x_tokens_canonical)
print("x_ids =", x_ids)
x_tokens_recovered = vocab.ids_to_words(x_ids)
x_tokens_recovered

Note the additional `<s>` and `</s>` tokens - these are used to represent the beginning and end of sentences. We'll make use of these in Assignment 3 and Assignment 4.

### Converting IDs to a feature vector

Many NLP models are designed to work directly with a sequence of word ids. However, for many standard machine learning models such as Naive Bayes, SVMs, or Logistic Regression, we need to convert this sequence to a fixed-length vector $x \in \mathbb{R}^d$. In the general case, we can define a set of feature extractors $f_i$ for $i = 0,1,\ldots,d-1$, such that $x_i = f_i([\mathtt{ids}])$.

The simplest way to do this is a bag-of-words model, in which we let the number of features be the size of our vocabulary ($d = |V|$), and we let each feature be a count of the number of times word $i$ appears in the sequence:

$$ f_i([\mathtt{ids}]) = \sum_{j = 0}^{n} \mathbf{1}[w_j = i] $$

We can do this in a very simple way by using the `collections.Counter` class:

In [8]:
print("Example, with words as keys:", collections.Counter(x_tokens_canonical))
x_fdict = collections.Counter(x_ids)
x_fdict

A common data format to use in machine learning applications is to transform this dictionary-like object that maps keys to values into a feature vector:

In [9]:
num_features = vocab.size  # one feature for each word
x_vector = [x_fdict.get(i, 0) for i in range(num_features)]
x_vector

If one has multiple examples, these are represented as multiple rows of these vectors stacked on top of one another (similar to the batching you did in assignment 2).  If $|V|$ is large and the text short, it is likely most of the elements of such a matrix are zero.  A memory optimization can be made by using a [sparse vector](https://docs.scipy.org/doc/scipy/reference/sparse.html) representation. (Or for more than one example, a sparse matrix.) You may have worked with these before, as they are the preferred input format for many `scikit-learn` ML routines.

The sparse matrix constructor requires three parallel lists: the row indices, the column indices, and the corresponding values.  Note that we have just a single row (we have only one example), so all the row indices are 0.

In [10]:
#  See https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
from scipy.sparse import csr_matrix 

row_indices = []
col_indices = []
values = []

# Construct three parallel lists as described above to satisfy the sparse matrix constructor.
for wordid, count in x_fdict.items():
    row_indices.append(0)       # only a single example, so row 0
    col_indices.append(wordid)  # column is word id
    values.append(count)        # value is count
x_sparse = csr_matrix((values, (row_indices, col_indices)),
                      shape=[1, vocab.size])
print("Non-zero values:")
print(x_sparse)
x_sparse

We've provided a helper function, `utils.id_lists_to_sparse_bow` that can handle this conversion over a whole dataset, and in most cases we'll handle this conversion for you in the starter code.